In [ ]:
!nvidia-smi

Wed Jun 17 12:58:42 2020       

+-----------------------------------------------------------------------------+

| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |

|-------------------------------+----------------------+----------------------+

| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |

| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |

|                               |                      |               MIG M. |

|===============================+======================+======================|

|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |

| N/A   45C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |

|                               |                      |                 ERR! |

+-------------------------------+----------------------+----------------------+

                                                                            

In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
drive.mount('/content/drive',force_remount=True)
%cd '/content/drive/My Drive/malaria_detection'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly



Enter your authorization code:

··········

Mounted at /content/drive

Mounted at /content/drive

/content/drive/My Drive/malaria_detection


In [ ]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh, init_git
password = 'dlpeD5580VAoErblTlkAD5MT0rGNEElsQjMb5NWMw'
ngrokToken = '1clMumj9JayqNG6nliy0OilP5dt_56pUjh2xf7YcbphYo6zgw'
launch_ssh(ngrokToken, password)
# %cd '/content/drive/My Drive/malaria_detection'

# # Optional: if you want to clone a github repository
# githubUrl = 'https://github.com/pokaxpoka/deep_Mahalanobis_detector'
# # githubUrl = 'https://github.com/i-gao/ood-mahalanobis'
# #githubUrl = 'https://github.com/djmMax/ood-mahalanobis'
# init_git(githubUrl)

Requirement already up-to-date: colab_ssh in /usr/local/lib/python3.6/dist-packages (0.2.58)

Successfully running 0.tcp.ngrok.io:18143

[Optional] You can also connect with VSCode SSH Remote extension using this configuration:



	Host google_colab_ssh

		HostName 0.tcp.ngrok.io

		User root

		Port 18143

	  


In [ ]:
import os
import numpy as np
import shutil
import random

# Creating Train / Val / Test folders (One time use)
def split_data(root_dir, classes_dir, val_ratio, test_ratio, random_seed=None):

  for cls in classes_dir:
      os.makedirs(root_dir +'/train' + cls, exist_ok=True)
      os.makedirs(root_dir +'/val' + cls, exist_ok=True)
      os.makedirs(root_dir +'/test' + cls, exist_ok=True)

      # Creating partitions of the data after shuffeling
      src = root_dir + cls # Folder to copy images from

      allFileNames = os.listdir(src)
      if random_seed != None :
        np.random.seed(random_seed)  #fix random
      np.random.shuffle(allFileNames)
      train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                                [int(len(allFileNames)* (1 - val_ratio - test_ratio)), 
                                                                int(len(allFileNames)* (1 - test_ratio))])


      train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
      val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
      test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

      print('Total images: ', len(allFileNames))
      print('Training: ', len(train_FileNames))
      print('Validation: ', len(val_FileNames))
      print('Testing: ', len(test_FileNames))

      # Copy-pasting images
      for name in train_FileNames:
          shutil.move(name, root_dir +'/train' + cls)

      for name in val_FileNames:
          shutil.move(name, root_dir +'/val' + cls)

      for name in test_FileNames:
          shutil.move(name, root_dir +'/test' + cls)

In [ ]:
# !wget ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
# !unzip cell_images.zip > /dev/null

In [ ]:
# !unzip '/content/drive/My Drive/malaria_detection/Images.zip' > /dev/null

## Detecting Out-of-Distribution Samples (Baseline and ODIN)

In [ ]:
# model: ResNet, in-distribution: CIFAR-10, gpu: 0
! cd deep_Mahalanobis_detector && python OOD_Baseline_and_ODIN.py --dataset malaria --net_type resnet18 --gpu 0

Namespace(batch_size=200, dataroot='./data', dataset='malaria', gpu=0, net_type='resnet18', num_classes=10, outf='./output/')

load model: resnet18

load target data:  malaria

data_root: ./data

Loaded 16535 / train

['Parasitized', 'Uninfected']

Loaded 8267 / val

['Parasitized', 'Uninfected']

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add is deprecated:

	add(Tensor input, Number alpha, Tensor other, *, Tensor out)

Consider using one of the following signatures instead:

	add(Tensor input, Tensor other, *, Number alpha, Tensor out)

/content/drive/My Drive/malaria_detection/deep_Mahalanobis_detector/lib_generation.py:247: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.

  outputs = model(Variable(tempInputs, volatile=True))

Temperature: 1 / noise: 0

data_root: ./data

Loaded 6613 / train

['Parasitized', 'Uninfected']

Loaded 6614 / val

['Parasitized', 'Uninfected']

Out-distribution: malaria

## Detecting Out-of-Distribution Samples (Mahalanobis detector)

### 1. Extract detection characteristics:

In [ ]:
# model: ResNet, in-distribution: CIFAR-10, gpu: 0
! cd deep_Mahalanobis_detector && python OOD_Generate_Mahalanobis.py --dataset malaria --net_type resnet18 --gpu 0

Namespace(batch_size=200, dataroot='./data', dataset='malaria', gpu=0, net_type='resnet18', num_classes=10, outf='./output/')

load model: resnet18

load target data:  malaria

data_root: ./data

Loaded 16535 / train

['Parasitized', 'Uninfected']

Loaded 8267 / val

['Parasitized', 'Uninfected']

get sample mean and covariance

/content/drive/My Drive/malaria_detection/deep_Mahalanobis_detector/lib_generation.py:69: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.

  data = Variable(data, volatile=True)



 Training Accuracy:(84.71%)



get Mahalanobis scores

Noise: 0.0

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add is deprecated:

	add(Tensor input, Number alpha, Tensor other, *, Tensor out)

Consider using one of the following signatures instead:

	add(Tensor input, Tensor other, *, Number alpha, Tensor out)

/content/drive/My Drive/malaria_detection/deep_Mahalanobis_detector/lib_generation.py:181

### 2. Train simple detectors:

In [ ]:
# model: ResNet
! cd deep_Mahalanobis_detector && python OOD_Regression_Mahalanobis.py --net_type resnet18

Namespace(net_type='resnet18')

In-distribution:  malaria

Out-of-distribution:  malaria2

in_distribution: malaria==========

out_distribution: malaria2

 TNR    AUROC  DTACC  AUIN   AUOUT 

100.00 100.00 100.00  99.99  99.99

Input noise: Mahalanobis_0.0




## Detecting Adversarial Samples (LID & Mahalanobis detector)

### 0. Generate adversarial samples:

In [ ]:
#!pip install 'https://download.pytorch.org/whl/cu75/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl'

In [ ]:
! l

In [3]:
# model: ResNet, in-distribution: CIFAR-10, adversarial attack: FGSM  gpu: 0
! python ADV_Samples.py --dataset malaria --net_type resnet18 --adv_type FGSM --batch_size 16 --gpu 0

python: can't open file 'ADV_Samples.py': [Errno 2] No such file or directory


### 1. Extract detection characteristics:

In [ ]:
# model: ResNet, in-distribution: CIFAR-10, adversarial attack: FGSM  gpu: 0
! cd deep_Mahalanobis_detector && python ADV_Generate_LID_Mahalanobis.py --dataset malaria --net_type resnet18 --adv_type FGSM --gpu 0

Namespace(adv_type='FGSM', batch_size=200, dataroot='./data', dataset='malaria', gpu=0, net_type='resnet18', num_classes=10, outf='./output/')

load model: resnet18

load target data:  malaria

data_root: ./data

Loaded 16535 / train

['Parasitized', 'Uninfected']

Loaded 8267 / val

['Parasitized', 'Uninfected']

tcmalloc: large alloc 1660624896 bytes == 0xa23b0000 @  0x7f5de8844b6b 0x7f5de8864379 0x7f5d9d78204e 0x7f5d9d783f4a 0x7f5dd66710c4 0x7f5de48be5d9 0x551365 0x5a9cbc 0x50a5c3 0x50bfb4 0x507d64 0x509042 0x5a4b71 0x5a4ec8 0x4e009e 0x50a5f1 0x50bfb4 0x507d64 0x588e5c 0x59fc4e 0x50d356 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x50ae13 0x634c82

tcmalloc: large alloc 1660624896 bytes == 0x106c0c000 @  0x7f5de8844b6b 0x7f5de8864379 0x7f5d9d78204e 0x7f5d9d783f4a 0x7f5dd66710c4 0x7f5de48be5d9 0x551365 0x5a9cbc 0x50a5c3 0x50bfb4 0x507d64 0x509042 0x5a4b71 0x5a4ec8 0x4e009e 0x50a5f1 0x50bfb4 0x507d64 0x588e5c 0x59fc4e 0x50d356 0x507d64 0x509a90 0x50a48d 0x5

### 2. Train simple detectors:

In [ ]:
# model: ResNet
python ADV_Regression.py --net_type resnet